## Part 2 : Spark ALS 

In this section ,we train an out of the box Collabortive filtering algorithm provided by spark know as Alternating Least Square (ALS) .This alogirthm is an implementation of Matrix Factorization techinique.
We fit this model on movielens 20 million ratings and evaluate its RMSE and MAE.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###Setup Spark


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirrors.spacedump.net/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz


In [0]:
!tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install  findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7.tgz"

In [0]:
import findspark
findspark.init('/content/spark-3.0.0-preview2-bin-hadoop2.7')


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Load data


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

ratings = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/drive/My Drive/ml-dataset/20m/ratings.csv")
print("Number of records : ",ratings.count())
  
  



Number of records :  20000263


### Explore dataset


In [0]:
ratings.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|         20000263|          20000263|          20000263|            20000263|
|   mean|69045.87258292554| 9041.567330339605|3.5255285642993797|1.1009179216771157E9|
| stddev|40038.62665316201|19789.477445413086| 1.051988919294227|1.6216942478273067E8|
|    min|                1|                 1|               0.5|           789652004|
|    max|           138493|            131262|               5.0|          1427784002|
+-------+-----------------+------------------+------------------+--------------------+



In [0]:

print('Number of  users: {}'.format(ratings.select('userId').distinct().count()))
print('Number of  movies: {}'.format(ratings.select('movieId').distinct().count()))
print('Number of movies with at least one rating higher than 3: {}'.format(ratings.filter('rating > 3').select('movieId').distinct().count()))

Number of  users: 138493
Number of  movies: 26744
Number of movies with at least one rating higher than 3: 22884


### Train model


In [0]:

#split dataset
(training, test) = ratings.randomSplit([0.80, 0.20])

#train ALS 
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop")
model = als.fit(training)



### Evaluate


In [0]:
# Evaluate 
predictions = model.transform(test)

evaluatorRmse = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluatorRmse.evaluate(predictions)
print("RMSE  = " + str(rmse))

evaluatorMae = RegressionEvaluator( metricName="mae",labelCol="rating",predictionCol="prediction")
mae = evaluatorMae.evaluate(predictions)
print("Mean absolute error = " + str(mae))

RMSE  = 0.8135797927665147
Mean absolute error = 0.6254400840948735


### Results
We obsere that the RMSE for the ALS model is 0.81 with MAE of 0.62 on the 20 million ratings dataset.


In [0]:
from prettytable import PrettyTable
import numpy as np
x = PrettyTable()

x.field_names = ["Metric", "Value"]

x.add_row(["Root Mean Square Error ", np.round(rmse,4)])
x.add_row([" Mean Square Error ", np.round(np.square(rmse),4)])
x.add_row(["Mean Absolute Error", np.round(mae,4)])

print(x)

+-------------------------+--------+
|          Metric         | Value  |
+-------------------------+--------+
| Root Mean Square Error  | 0.8136 |
|    Mean Square Error    | 0.6619 |
|   Mean Absolute Error   | 0.6254 |
+-------------------------+--------+


### Next
IN next part we train our autoencoder models and compare results with ALS .


